In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
import os
from src.training_utils import train_evaluate, predict_metrics
from src.BertCLF import BertCLF
from transformers import AutoModel
from transformers import AutoTokenizer
import torch
import torch.optim as optim
import torch.nn as nn
import json
from src.preparing_data_utils import prepare_data, prepare_data_notebook, prepare_dataset
import pandas as pd

In [17]:
config = dict(
    transformer_model = dict(
        model = "cointegrated/rubert-tiny",
        path_to_state_dict = False,
        device = 'cuda',
        dropout = 0.2,
        tiny_bert = True, 
        learning_rate = 1e-6,
        batch_size = 8,
        shuffle = True,
        maxlen = 512,
    ),
    data = dict(
        train_data_path = "../../gvk_dnie_one_list.xlsx",
        test_data_path = None,
        text_column = "Комментарий",
        target_column = "target",
        random_state = 42,
        test_size = 0.3,
    ),
    training = dict (
    num_epochs = 20,
    average_f1 = 'macro',
    output_dir = "../results/"
    )
)

In [19]:
os.makedirs(config['training']['output_dir'], exist_ok=True)

In [6]:
device = torch.device(config['transformer_model']['device'])
tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=config['transformer_model']["model"]
    )
model_bert = AutoModel.from_pretrained(
    pretrained_model_name_or_path=config['transformer_model']["model"]
).to(device)

Here you can either use `prepare_data` function if you want just pass the
path to your data or you can pass the data itself in the form of pandas dataframe. In the latter case
you should use `prepare_data_notebook` function

In [7]:
id2label, train_texts, valid_texts, train_targets, valid_targets = prepare_data(config=config)

In [8]:
df = pd.read_excel("../../gvk_dnie_one_list.xlsx")

id2label, train_texts, valid_texts, train_targets, valid_targets = prepare_data_notebook(
    config=config,
    train_df=df
)

/home/user/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
model = BertCLF(
    pretrained_model=model_bert,
    tokenizer=tokenizer,
    id2label=id2label,
    dropout=config['transformer_model']['dropout'],
    tiny=config['transformer_model']['tiny_bert'],
    device=device     
    )

In [10]:
model = model.to(device)

In [11]:
optimizer = optim.Adam(model.parameters(), lr=float(config['transformer_model']['learning_rate']))
criterion = nn.NLLLoss()

training_generator, valid_generator = prepare_dataset(
    tokenizer=tokenizer,
    train_texts=train_texts,
    train_targets=train_targets,
    valid_texts=valid_texts,
    valid_targets=valid_targets,
    config=config
)

In [13]:
model = train_evaluate(
    model=model,
    training_generator=training_generator,
    valid_generator=valid_generator,
    criterion=criterion,
    optimizer=optimizer,
    num_epocs=config['training']['num_epochs'],
    average=config['training']['average_f1']
)

==== Epoch 1 out of 20 ====


Evaluating loop: 100%|██████████| 57/57 [00:00<00:00, 286.20it/s]


Train F1: 0.06051076238576238
Eval F1: 0.07499860763018658

==== Epoch 2 out of 20 ====


Evaluating loop: 100%|██████████| 57/57 [00:00<00:00, 259.53it/s]


Train F1: 0.1008680054513388
Eval F1: 0.12719808781212288

==== Epoch 3 out of 20 ====


...

Evaluating loop: 100%|██████████| 57/57 [00:00<00:00, 93.48it/s] 


Train F1: 0.40402298599268294
Eval F1: 0.41354706209969366




Computing final metrics...: 100%|██████████| 57/57 [00:00<00:00, 149.03it/s]

                                          precision    recall  f1-score   support

                             КОМПЕНСАЦИЯ       0.62      0.84      0.71       128
                      ПРОБЛЕМЫ С ЛИМИТОМ       0.00      0.00      0.00        37
                      Платформа СберДруг       0.55      0.52      0.53        93
                          Положительный        0.00      0.00      0.00         7
Процесс неизвестен, не дали использовать       0.00      0.00      0.00        30
                            СОГЛАСОВАНИЕ       0.00      0.00      0.00        33
                 ТАКСИ, ВОДИТЕЛИ, МАШИНЫ       0.54      0.82      0.65       125

                                accuracy                           0.57       453
                               macro avg       0.24      0.31      0.27       453
                            weighted avg       0.44      0.57      0.49       453




/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
predict_metrics(model=model, iterator=valid_generator)

Computing final metrics...: 100%|██████████| 57/57 [00:00<00:00, 150.58it/s]

                                          precision    recall  f1-score   support

                             КОМПЕНСАЦИЯ       0.62      0.84      0.71       128
                      ПРОБЛЕМЫ С ЛИМИТОМ       0.00      0.00      0.00        37
                      Платформа СберДруг       0.55      0.52      0.53        93
                          Положительный        0.00      0.00      0.00         7
Процесс неизвестен, не дали использовать       0.00      0.00      0.00        30
                            СОГЛАСОВАНИЕ       0.00      0.00      0.00        33
                 ТАКСИ, ВОДИТЕЛИ, МАШИНЫ       0.54      0.82      0.65       125

                                accuracy                           0.57       453
                               macro avg       0.24      0.31      0.27       453
                            weighted avg       0.44      0.57      0.49       453




/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
torch.save(model.state_dict(), os.path.join(config['training']['output_dir'], "model"))
with open(os.path.join(config['training']['output_dir'], 'label_mapper.json'), mode='w', encoding='utf-8') as f:
        json.dump(model.mapper, f, indent=4, ensure_ascii=False)